In [1]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.utilities           import percentError
from pybrain.datasets import SupervisedDataSet
from pybrain.datasets            import ClassificationDataSet
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn import tree
import getfeat
%matplotlib inline

In [2]:
def part(Y,partition):
    
    Y_q=np.zeros(len(Y))
    i=0
    for y in Y:
        k=0
        score=-1
        for p in partition:
            if y in p:
                score=k
                break
            k+=1
        if score==-1:
            score=len(partition)
        Y_q[i]=score
        i+=1
    return Y_q

In [17]:
partitions1=[[[0],[1],[2,3],[4,5]],
             [[0],[1,2],[3,4]],
             [[0,1],[2,3]]
            ]
Partitions=[partitions1]
for j in [2,3,4,5,6]:
    Q=getfeat.get_composed_feat_h(j)
    Y=Q[1]
    H=np.histogram(Y,list(range(int(max(Y))+1)))
    partitions=[]

    for D in [5,4,3]:
        start=0
        partition=[]
        for c in range(D-1):
            s=0
            for i in range(start,len(H[0])):
                s+=H[0][i]
                if s>len(Y)/float(D):
                    break
            partition.append(list(range(start,i+1)))
            start=i+1
        partitions.append(partition)
    Partitions.append(partitions)

In [18]:
for partition in Partitions:
    for p in partition:
        print(p)
        print '----'
    print ('****')

[[0], [1], [2, 3], [4, 5]]
----
[[0], [1, 2], [3, 4]]
----
[[0, 1], [2, 3]]
----
****
[[0, 1], [2, 3], [4, 5, 6], [7, 8, 9, 10]]
----
[[0, 1, 2], [3, 4, 5], [6, 7, 8, 9, 10]]
----
[[0, 1, 2], [3, 4, 5, 6]]
----
****
[[0, 1, 2], [3, 4, 5], [6, 7, 8, 9], [10, 11, 12, 13, 14]]
----
[[0, 1, 2, 3], [4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14]]
----
[[0, 1, 2, 3], [4, 5, 6, 7, 8, 9]]
----
****
[[0, 1, 2, 3], [4, 5, 6], [7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17]]
----
[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17]]
----
[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11, 12, 13, 14]]
----
****
[[0, 1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15], [16, 17, 18, 19, 20, 21]]
----
[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20]]
----
[[0, 1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
----
****
[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24, 25, 26, 27]]
----
[[0, 1, 2, 3, 4, 5, 6, 7], 

In [ ]:
n_feat=1321
n_est=200
klf=tree.DecisionTreeClassifier(max_depth=9)
clfs=[tree.DecisionTreeClassifier(),tree.DecisionTreeClassifier(min_samples_split=16),tree.DecisionTreeClassifier(max_depth=6)]
clfs+=[RandomForestClassifier(n_estimators=n_est, max_features=i,max_depth=9) for i in range(1,n_feat)]
clfs+=[BaggingClassifier(base_estimator=klf,max_features=i,n_estimators=n_est) for i in range(1,n_feat)]
clfs+=[GradientBoostingClassifier(n_estimators=n_est, learning_rate=1.0,max_depth=9, random_state=0)]
clfs+=[AdaBoostClassifier(base_estimator=klf)]

In [55]:
n_feat=1321
n_est=200
#n_feat_selection=[i*10 for i in range (1,n_feat/10+1)]
#n_feat_selection=[5]+[i*10 for i in range(1,10)]+[100+i*20 for i in range(200/20)]+[300+i*30 for i in range(300/30)]+[600+100*i for i in range(500/100)]+[1321]
#n_feat_selection=[5]+[i*20 for i in range(1,5)]+[100+i*40 for i in range(200/40)]+[300+i*60 for i in range(300/60)]+[600+250*i for i in range(500/250)]+[1321]
n_feat_selection=[5]+[i*40 for i in range(1,2)]+[100+i*50 for i in range(200/50)]+[300+i*100 for i in range(300/100)]+[600+250*i for i in range(500/250)]+[1321]
klf=tree.DecisionTreeClassifier(max_depth=9)
clfs=[tree.DecisionTreeClassifier(),tree.DecisionTreeClassifier(min_samples_split=16),tree.DecisionTreeClassifier(max_depth=6)]
clfs+=[RandomForestClassifier(n_estimators=n_est, max_features=i,max_depth=9,n_jobs=-1) for i in n_feat_selection]
clfs+=[BaggingClassifier(base_estimator=klf,max_features=i,n_estimators=n_est,n_jobs=-1) for i in n_feat_selection]
clfs+=[GradientBoostingClassifier(n_estimators=n_est, learning_rate=1.0,max_depth=9, random_state=0)]
clfs+=[AdaBoostClassifier(base_estimator=klf)]

In [56]:
names=['tree','tree_min_slit_16','tree_max_depth_6']
names+=['RandomForest_'+str(i)+'_f' for i in  n_feat_selection]
names+=['Bagging_'+str(i)+'_f' for i in  n_feat_selection]
names+=['GradientBoosting']
names+=['AdaBoost']

In [57]:
test_sizes=[120,70,50,35,30,25]

In [58]:
len(clfs), len(names)

(29, 29)

In [59]:
def evaluate_clf(clf,X,Y_q,features,test_size):
    I=np.random.permutation(len(X))
    M=[]
    S=[]
    Acc=[]
    P=[]
    R=[]
    Conf=[]
    for i in range(len(X)/test_size-1):
        I_te=list(range(test_size*i,test_size*(i+1)))
        I_tr=list(range(0,test_size*i))+list(range(test_size*(i+1),len(I)))
        train=I[I_tr]
        test=I[I_te]

        XX = np.array([d[features] for d in X[train]])
        y=Y_q[train] 
        clf = clf.fit(XX, y)
        true=np.array([int(Y_q[t]) for t in test])
        pred=np.array([int(clf.predict([X[t][features]])[0]) for t in test])
        conf=confusion_matrix(true,pred) 
        recall=[0 if not conf[i,i] else float(conf[i,i])/sum(conf[i,:]) for i in range(len(conf[:,0]))]
        precision=[0 if not conf[i,i] else float(conf[i,i])/sum(conf[:,i]) for i in range(len(conf[:,0]))]
        
        Conf.append(conf)
        M.append(np.average(np.abs(true-pred)))
        S.append(np.std(true-pred))
        Acc.append(accuracy_score(true,pred))
        R.append(recall)
        P.append(precision)
    return[M,S,Acc,R,P,Conf]

In [60]:
def writeout(res,partition,aggr,features,names,filename,i):
    f=open(filename,'a')
    f.write(str(i)+';'+names[i]+';'+str(aggr)+';'+str(partition)+';'+str(features)+';'+str(res[0])+';'+str(res[1])+';'+str(res[2])+';'+str(res[3])+';'+str(res[4])+';\n')
    f.close()
def writeout_c(conf,partition,aggr,features,names,filename,i):
    f=open(filename,'a')
    f.write(str(i)+';'+names[i]+';'+str(aggr)+';'+str(partition)+';'+str(features)+';\n')
    f.write(str(conf)+'\n')
    f.close()

In [61]:
filename='report_acc_2'
filename_c='report_conf_2'
f=open(filename,'a')
#f.write('n_classf;name_classf;h_aggregation;risk_bands;features;avg_distance_from_true;std_distance_from_true;accuracy;recalls;precisions\n')
f.close()
f=open(filename_c,'a')
#f.write('n_classf;name_classf;h_aggregation;risk_bands;features;confusion\n')
f.close()
for aggr in range(2,7):
    test_size=test_sizes[aggr-1]
    if aggr==1:
        Q=getfeat.get_composed_feat_1h()
    else:
        Q=getfeat.get_composed_feat_h(aggr)
    X=Q[0].T
    Y=Q[1]
    if aggr==2:
        for partition in Partitions[aggr-1][1:]:
            Y_q=part(Y,partition)
            features=list(range(n_feat))
            for i in range(len(clfs)):
                res=evaluate_clf(clfs[i],X,Y_q,features,test_size)
                writeout(res,partition,aggr,features,names,filename,i)
                writeout_c(res[-1],partition,aggr,features,names,filename_c,i)
    else:
        for partition in Partitions[aggr-1]:
            Y_q=part(Y,partition)
            features=list(range(n_feat))
            for i in range(len(clfs)):
                res=evaluate_clf(clfs[i],X,Y_q,features,test_size)
                writeout(res,partition,aggr,features,names,filename,i)
                writeout_c(res[-1],partition,aggr,features,names,filename_c,i)

In [37]:
n_feat_selection

[10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 120,
 140,
 160,
 180,
 200,
 220,
 240,
 260,
 280,
 300,
 330,
 360,
 390,
 420,
 450,
 480,
 510,
 540,
 570,
 600,
 700,
 800,
 900,
 1000,
 1321]